In [54]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib; matplotlib.rcParams['figure.figsize'] = (15,3)

In [55]:
import numpy as np

In [56]:
# Reload all packages - make debugging easier
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import tensorflow as tf
from nnmf_svi_eddie import save_graph_parameters

In [58]:
from nnmf_svi_eddie import NNMF

---

This is just for debugging. Use `hyperparam_search.py` instead.

```
[x] Split to train/validation
[x] MSE
[x] Save weights
[ ] Visualize results

Next: Find out what dataset to train on, and test MSE on validation set.
```

---

In [59]:
def get_hypers_config(seed):
    import random
    random.seed(seed)
    
    D = random.randint(5,100)
    Dp = random.randint(5,100)
    
    nn_hidden_layer_dims = []
    n_layers = random.randint(1,5)
    for l in range(n_layers):
        nn_hidden_layer_dims.append(random.randint(5,100))
        
    batch_size = random.randint(100,400)
    n_samples = random.choice([0,10,100])
    
    pZ_prior_stddev = random.randrange(200) / 100 # 0 to 2
    pR_stddev = random.randrange(200) / 100 # 0 to 2
    
    nn_W_init_mean = 0.
    nn_W_init_stddev = random.randrange(200) / 100 # 0 to 2
    nn_b_init_mean = 0.
    nn_b_init_stddev = random.randrange(200) / 100 # 0 to 2

    optimizer = 'adam'
    lr_init = random.choice([0.01,0.1,1.0])
    lr_decay_steps = random.choice([100,200,300])
    lr_decay_rate = random.choice([0.9,0.95,0.99])

    return {
        'D': D,
        'Dp': Dp,
        'nn_hidden_layer_dims': nn_hidden_layer_dims,
        'batch_size': batch_size,
        'n_samples': n_samples,
        'pZ_prior_stddev': pZ_prior_stddev,
        'pR_stddev': pR_stddev,
        'nn_W_init_mean': nn_W_init_mean,
        'nn_W_init_stddev': nn_W_init_stddev,
        'nn_b_init_mean': nn_b_init_mean,
        'nn_b_init_stddev': nn_b_init_stddev,
        'optimizer': optimizer,
        'lr_init': lr_init,
        'lr_decay_steps': lr_decay_steps,
        'lr_decay_rate': lr_decay_rate
    }

def save_output_csv(fname, data_list):    
    import csv
    with open(fname, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(data_list)

In [63]:
_SESS = None
def get_new_session():
    global _SESS
    if _SESS is not None:
        _SESS.close()
    _SESS = tf.Session()
    return _SESS
    
def hypersearch(folder, model_name, dataset_name, seed, n_iter, R, train_mask, valid_mask, verbose=False):
    import os
    os.makedirs(folder, exist_ok=True)

    csv_output = "{}/{}_{}_{}_{}.csv".format(folder, model_name, dataset_name, seed, n_iter)
    mdl_output = "{}/{}_{}_{}_{}.pkl".format(folder, model_name, dataset_name, seed, n_iter)
    
    if os.path.exists(csv_output):
        print("Skip #{}".format(seed))
    else:
        hypers = get_hypers_config(seed)
        if verbose: print(hypers)

        tf.reset_default_graph()

        sess = get_new_session()
        with sess.as_default():
            model = NNMF(ratings_matrix=R, **hypers)
            losses = model.train(train_mask, n_iter=n_iter, verbose=verbose)
            if verbose:
                plt.plot(losses)
                plt.show()

            # Evaluation
            if verbose: print('Evaluating...')
            def get_mse(mask):
                # We evaluate 10 draws at a time so everything fits into memory.
                results_batches = []
                for _ in range(10): # 100 draws total
                    idx_i_all, idx_j_all = np.where(mask)
                    feed_dict = {
                        model.test_idx_i: idx_i_all,
                        model.test_idx_j: idx_j_all,
                        model.n_test_samples: 10
                    }
                    results_batch = np.squeeze(sess.run(model.sample_rhats, feed_dict))
                    results_batches.append(np.mean(results_batch, axis=0))
                results_batches = np.array(results_batches)
                results = np.mean(results_batches, axis=0)
                mse = np.mean(np.square(results - R[idx_i_all, idx_j_all]))
                return mse
            
            train_mse = get_mse(train_mask)
            valid_mse = get_mse(valid_mask)

            # Output
            data_list = [model_name, dataset_name, seed, n_iter, losses[-1], train_mse, valid_mse]
            if verbose: print(data_list)
            save_output_csv(csv_output, data_list)
            save_graph_parameters(mdl_output)
            if verbose: print("Done #{}".format(seed))

# Run it:

In [71]:
# Load your data here:
import sys 
sys.path.append('..')

# Soon's code provides us with training matrix
from sclrecommender.parser import MovieLensParser
seedNum = 196
np.random.seed(seedNum)
dataDirectory = "../ml-100k"
mlp = MovieLensParser(dataDirectory)
R = mlp.getRatingMatrixCopy()

# ---
import random
some_really_random_number = random.randint(0,999999999)
# ---

mask = R>0

np.random.seed(1337) # Set a fixed seed, so we get a fixed mask.
TRAIN_MASK = (np.random.binomial(1, 0.9, size=R.shape)) & mask
VALID_MASK = (1-TRAIN_MASK) & mask

np.random.seed(some_really_random_number) # Go back to "true" randomness.

In [73]:
MODEL_NAME = 'NNMF'
DATASET_NAME = 'TEST'
SEED = np.random.randint(999999999)
N_ITER = 200
OUTPUT_FOLDER = 'hypersearch_v1'

print(SEED)
%time hypersearch(OUTPUT_FOLDER, MODEL_NAME, DATASET_NAME, SEED, N_ITER, R, TRAIN_MASK, VALID_MASK, verbose=True)

307998201
{'D': 56, 'Dp': 80, 'nn_hidden_layer_dims': [5, 30, 49], 'batch_size': 308, 'n_samples': 100, 'pZ_prior_stddev': 1.97, 'pR_stddev': 1.41, 'nn_W_init_mean': 0.0, 'nn_W_init_stddev': 0.36, 'nn_b_init_mean': 0.0, 'nn_b_init_stddev': 0.35, 'optimizer': 'adam', 'lr_init': 1.0, 'lr_decay_steps': 100, 'lr_decay_rate': 0.9}
 140/1000 [ 14%] ████                           ETA: 1083s | Loss: 2566672.250

KeyboardInterrupt: 

---